In [ ]:
import cv2
import scipy.stats.qmc as qmc
import scipy
import matplotlib.pyplot as plt 
import numpy as np
import pickle
from collections import Counter
import random
from manim import *

In [ ]:
with open('../out/strokes.pkl', 'rb') as inp:
    brushstrokes = pickle.load(inp)
with open('../out/U.pkl', 'rb') as inp:
    U = np.real(pickle.load(inp))
with open('../out/V.pkl', 'rb') as inp:
    V = np.real(pickle.load(inp))

    
sz = np.shape(brushstrokes[0])


U = np.random.random(sz)
V = np.random.random(sz)

im = cv2.imread('../res/starry_night_crop.jpg')
im = cv2.colorChange(im, None, cv2.COLOR_RGB2BGR)

In [ ]:
def backtrace(coordinates_x, coordinates_y, velocity_x_prev, velocity_y_prev, dt, aspect_ratio):
  velocity_x_prev = np.copy(velocity_x_prev)
  velocity_y_prev = np.copy(velocity_y_prev)
  nx = np.shape(coordinates_x)[1]
  ny = np.shape(coordinates_x)[0]
  #First, get the backtraced coordinates (if you follow the streamline
  #back, where do we go?)
  backtraced_coordinates_x = coordinates_x - dt * velocity_x_prev
  backtraced_coordinates_y = coordinates_y - dt * velocity_y_prev

  #The reference tells us the matrix index of the matrix we need to select
  backtraced_reference_x = backtraced_coordinates_x / aspect_ratio * nx
  backtraced_reference_x_low = np.floor(backtraced_reference_x)
  backtraced_reference_x_high = np.ceil(backtraced_reference_x)
  linear_const_x = backtraced_reference_x - backtraced_reference_x_low
  
  # backtraced_reference_x_low = np.clip(backtraced_reference_x_low, 0, nx-1)
  # backtraced_reference_x_high = np.clip(backtraced_reference_x_high, 0, nx-1)
  backtraced_reference_x_low = backtraced_reference_x_low % nx
  backtraced_reference_x_high = backtraced_reference_x_high % ny

  backtraced_reference_y = backtraced_coordinates_y * ny
  backtraced_reference_y_low = np.floor(backtraced_reference_y)
  backtraced_reference_y_high = np.ceil(backtraced_reference_y)
  linear_const_y = backtraced_reference_y - backtraced_reference_y_low

  # backtraced_reference_y_low = np.clip(backtraced_reference_y_low, 0, ny-1)
  # backtraced_reference_y_high = np.clip(backtraced_reference_y_high, 0, ny-1)

  backtraced_reference_y_low = backtraced_reference_y_low % ny
  backtraced_reference_y_high = backtraced_reference_y_high % ny

  #Need to reshape the velocity matrix to account for the fact that we can
  #only index a vector not a matrix
  velocity_x_row = np.reshape(velocity_x_prev,[np.size(velocity_x_prev)])
  velocity_y_row = np.reshape(velocity_y_prev,[np.size(velocity_y_prev)])

  #Need to amend the referencing to account for the fact it is a vector not
  #a matrix.
  low_low_index = np.int32((backtraced_reference_x_low)+(backtraced_reference_y_low)*nx)
  low_high_index = np.int32((backtraced_reference_x_low)+(backtraced_reference_y_high)*nx)
  high_low_index = np.int32((backtraced_reference_x_high)+(backtraced_reference_y_low)*nx)
  high_high_index = np.int32((backtraced_reference_x_high)+(backtraced_reference_y_high)*nx)

  #Complete interpolation for backtracing
  velocity_x = velocity_x_row[low_low_index]* (1-linear_const_x)*(1-linear_const_y) + velocity_x_row[low_high_index]* (1-linear_const_x)*(linear_const_y) + velocity_x_row[high_low_index]* (linear_const_x)*(1-linear_const_y) + velocity_x_row[high_high_index]* (linear_const_x)*(linear_const_y)
  velocity_y = velocity_y_row[low_low_index]* (1-linear_const_x)*(1-linear_const_y) + velocity_y_row[low_high_index]* (1-linear_const_x)*(linear_const_y) + velocity_y_row[high_low_index]* (linear_const_x)*(1-linear_const_y) + velocity_y_row[high_high_index]* (linear_const_x)*(linear_const_y)

  return (velocity_x, velocity_y)

In [ ]:
def zero_mean_velocity(velocity_x, velocity_y):
  velocity_x = np.copy(velocity_x)
  velocity_y = np.copy(velocity_y)
  #Subtract Mean
  velocity_x = velocity_x - np.mean(velocity_x)
  velocity_y = velocity_y - np.mean(velocity_y)

  return (velocity_x, velocity_y)

In [ ]:
def diffuse_incompressible(velocity_x, velocity_y, decay, normalized_wavenumbers_x, normalized_wavenumbers_y):
    velocity_x = np.copy(velocity_x)
    velocity_y = np.copy(velocity_y)

    n_points_y = np.shape(velocity_x)[0]
    n_points_x = np.shape(velocity_x)[1]

    #Transform Into Fourier
    velocity_x_fft = np.fft.fft2(velocity_x)
    velocity_y_fft = np.fft.fft2(velocity_y)
    
    #Low Pass Filter
    velocity_x_fft = velocity_x_fft * decay
    velocity_y_fft = velocity_y_fft * decay
    
    #Compute Pseudo Pressure
    pressure_fft = velocity_x_fft * normalized_wavenumbers_x + velocity_y_fft * normalized_wavenumbers_y
    
    #Project Velocities to be Incompressible
    velocity_x_fft = velocity_x_fft - pressure_fft * normalized_wavenumbers_x
    velocity_y_fft = velocity_y_fft - pressure_fft * normalized_wavenumbers_y
    #Transform Into Spatial
    velocity_x = np.real(np.fft.ifft2(velocity_x_fft, [n_points_y, n_points_x]));
    velocity_y = np.real(np.fft.ifft2(velocity_y_fft, [n_points_y, n_points_x]));

    return (velocity_x, velocity_y)

In [ ]:
#HYPERPARAMETERS
nu = 1/50
dt = 0.01
n_iter = 10000

n_points_y = sz[0]
n_points_x = sz[1]

aspect_ratio = n_points_x/n_points_y
coordinates_x = np.ones([n_points_y,1]) @ np.array([np.arange(start = 0, stop = aspect_ratio,step = aspect_ratio/n_points_x)])#linspace(0,aspect_ratio, n_points_x);
coordinates_y = np.array([np.arange(0,1,1/n_points_y)]).T * np.ones([1,n_points_x])

wavenumbers_1d_x = np.fft.fftfreq(n_points_x)
wavenumbers_1d_y = np.fft.fftfreq(n_points_y)
#wavenumbers_1d_x = [0:((n_points_x - rem(n_points_x,2))/2-1), -((n_points_x - rem(n_points_x,2))/2):-1];
n_fft_points_x = np.shape(wavenumbers_1d_x)[0]
#wavenumbers_1d_y = [0:((n_points_y - rem(n_points_y,2))/2-1), -((n_points_y - rem(n_points_y,2))/2):-1];
n_fft_points_y = np.shape(wavenumbers_1d_y)[0]

wavenumbers_x = np.ones([n_fft_points_y,1]) @ np.array([wavenumbers_1d_x])
wavenumbers_y = np.array([wavenumbers_1d_x]).T @ np.ones([1,n_fft_points_x])
wavenumbers_x_squared = wavenumbers_x * wavenumbers_x
wavenumbers_y_squared = wavenumbers_y * wavenumbers_y
wavenumbers_norm = np.sqrt(wavenumbers_x_squared + wavenumbers_y_squared)

decay = np.exp(-dt * nu * wavenumbers_norm * wavenumbers_norm)
wavenumbers_norm[wavenumbers_norm==0] = 1
normalized_wavenumbers_x = wavenumbers_x / wavenumbers_norm
normalized_wavenumbers_y = wavenumbers_y / wavenumbers_norm

velocity_x = np.zeros([n_points_y, n_points_x])
velocity_y = np.zeros([n_points_y, n_points_x])

velocity_x_prev = np.zeros([n_points_y, n_points_x])
velocity_y_prev = np.zeros([n_points_y, n_points_x])
velocity_x_prev = U
velocity_x_prev[np.isnan(velocity_x_prev)] = 0
velocity_y_prev = V
velocity_y_prev[np.isnan(velocity_y_prev)] = 0

In [ ]:

def curl_fft_UV(U,V):
    d_u_d_y_fft = 1j * wavenumbers_y * np.fft.fft2(U)
    d_v_d_x_fft = 1j * wavenumbers_x * np.fft.fft2(V)
    curl_fft = d_v_d_x_fft - d_u_d_y_fft
    return curl_fft
def curl_diagram(curl_fft):
    # print(curl_fft.shape)
    curl = np.fft.ifft2(curl_fft, sz)
    curl = np.real(curl)
    plt.imshow(curl, cmap = plt.cm.coolwarm)

    plt.colorbar()
    plt.show()
curl_diagram(curl_fft_UV(velocity_x_prev, velocity_y_prev))

In [ ]:

curl_diagram(curl_fft_UV(velocity_x_prev, velocity_y_prev))
for iter in range(n_iter):
    velocity_x, velocity_y = backtrace(coordinates_x, coordinates_y, velocity_x_prev, velocity_y_prev, dt, aspect_ratio)

    velocity_x, velocity_y = zero_mean_velocity(velocity_x, velocity_y)

    velocity_x, velocity_y = diffuse_incompressible(velocity_x, velocity_y, decay, normalized_wavenumbers_x, normalized_wavenumbers_y)

    velocity_x, velocity_y = zero_mean_velocity(velocity_x, velocity_y)

    #Progress in Time
    velocity_x_prev = np.copy(velocity_x)
    velocity_y_prev = np.copy(velocity_y)
    if (iter%100==0):
        curl_diagram(curl_fft_UV(velocity_x, velocity_y))



    


In [ ]:
class ContinuousMotion(Scene):
    def construct(self):
        func = lambda pos: np.sin(pos[0] / 2) * UR + np.cos(pos[1] / 2) * LEFT
        stream_lines = StreamLines(func, stroke_width=2, max_anchors_per_line=30)
        self.add(stream_lines)
        stream_lines.start_animation(warm_up=False, flow_speed=1.5)
        self.wait(stream_lines.virtual_time / stream_lines.flow_speed)

In [ ]:
a = np.max([3,4,-2,4])
a += np.min(a)
a /= np.max(a)
a